In [ ]:
import time
import schedule
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import smtplib
from email.message import EmailMessage
from datetime import date

def formolvas():
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('API.json',
                                                                   scope)
    gc = gspread.authorize(credentials)

    spread = gc.open("Nameday Alert")

    f_nev = pd.DataFrame(spread.worksheet("lap").get_all_values()).pipe(
        lambda df: df.rename(columns = df.iloc[0])).drop(0).reset_index(drop = True)
    return f_nev
   


def mainap():
    today = date.today()
    d1 = today.strftime("%d/%m")
    return d1

    
    
def scrape():
    names_url = "https://hu.wikipedia.org/wiki/Magyar_névnapok_listája_dátum_szerint"
    resp = requests.get(names_url)
    soup = BeautifulSoup(resp.content)
    table = soup.find("table")
    months = table.find_all("td")

    list_of_dicts = []
    month_number = 0
    for month in months:
        month_number += 1
        name_of_month = month.find("b").text
        month_days = month.find_all("li")
        actual_day = 0
        for month_day in month_days:
            actual_day += 1
            day_names = month_day.find_all("a")
            name_rank = 0
            for day_name in day_names:
                name_rank += 1
                list_of_dicts.append({
                    "month_number": month_number,
                    "month_name": name_of_month,
                    "day_number": actual_day,
                    "name_rank": name_rank,
                    "name":day_name.text
                })
    df = pd.DataFrame(list_of_dicts)
    df.to_csv('names_dates.csv')
    
    
    
def emailkuldes(d1, f_nev):
    
    csv_r = pd.read_csv("names_dates.csv") 
    
    s_nev = csv_r.loc[(csv_r['month_number'] == int(d1[-2:])) & (csv_r['day_number'] == int(d1[:2]))]['name'].tolist()

    for index, row in f_nev.iterrows():
        friends = row["Szeretteid keresztneve? (vesszővel, space-el elválasztva - pl. Sándor, Dóra, Gabriella)"].split(', ')
        names = set(s_nev).intersection(set(friends))
        if names != set():
            msg = EmailMessage()
            msg.set_content(('Ma van %s névnapja, köszöntsd fel!'  % ', '.join(names)).encode("UTF-8"),
                            maintype="text", subtype="plain")

            msg['Subject'] = 'Ne feledd!'
            msg['From'] = 'nomoreforgottennameday@gmail.com'
            msg['To'] = row['E-mail-cím']

            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.login("nomoreforgottennameday@gmail.com", "Makarenko6")
            server.send_message(msg)
            server.quit()
              
        
        
def minden():
    f_nev = formolvas()
    d1 = mainap()
    scrape()

    schedule.every().day.at("22:34").do(emailkuldes, d1, f_nev)
    while True:
        schedule.run_pending()
        time.sleep(60)
        
        
minden()